# Quantum Detector Tomography

This notebook contains examples for using QDT module with Qiskit.

In [1]:
import povmtools
import ancillary_functions as anf
import numpy as np

from qiskit import IBMQ, Aer, execute
from qiskit.providers.aer import noise

from quantum_tomography_qiskit import detector_tomography_circuits
from DetectorTomographyFitter import DetectorTomographyFitter

## Single Qubit detector tomography example

In below example it's shown how to implement single-qubit Quantum Detector Tomography (QDT) using our module. 

First we need to create quantum circuits which will be used to implement QDT. To do so, need to define indices of qubits on which we want to perform tomography. In this example we choose single qubit with label 3.

In [2]:
# choose qubit indices
test_qubit_index = [3]

We will also need kets upon which the tomography shall be based. In this scenario we shall use overcomplete set of Pauli's eigen states. Our module provides them in following way, using povmtools module.

In [3]:
test_probe_kets = povmtools.pauli_probe_eigenkets

Now we call detector_tomography_circuits method, which will generate desired circuits. 

In [4]:
test_circuits = detector_tomography_circuits(test_qubit_index, test_probe_kets)

### User-defined probe states

User can define list of single-qubit state vectors which will be used to perform tomography of the detector (in the case of multi-qubit tomography, only single-qubit set is required - the multi-qubit states are constructed from proper tensor products of those, see multi-qubit QDT example below).

In order for tomography to work properly, the set of passed qubit state vectors, when mapped onto quantum states (i.e., density matrices), must span the space of Hermitian matrices. In other words, it must form an operator basis (it might be an overcomplete basis).

If no probe_kets are provided then overcomplete set of Pauli's eigenstates is used:

{|0>, |1>, |X+>, |X->, |Y+>, |Y->}

### Implementation of QDT

After defining tomography circuits, we need to implement them. 

To do so, we use Qiskit simulator in the following way.

In [5]:
backend = Aer.get_backend('qasm_simulator')

#define number of measurement repetitions
shots_number = 2000

QDT_job = execute(test_circuits, backend=backend, shots=shots_number)

results = QDT_job.result()

With probe kets and job results we can begin to perform QDT. We start with instantiating DetectorTomographyFitter class object.

In [6]:
DTF = DetectorTomographyFitter()

This class contains method called get_maximum_likelihood_povm_estimator which returns the classical description of our detector. This methods requires results and probe kets as arguments. We call the method and print the results.

In [7]:
#we pass list of results (in this case singular) to the method
ml_povm_estimator = DTF.get_maximum_likelihood_povm_estimator([results], test_probe_kets)

for m_i in ml_povm_estimator:
    print(m_i)

[[9.99827714e-01-4.83338176e-20j 1.25004513e-02-3.99939602e-03j]
 [1.25004513e-02+3.99939602e-03j 1.72286133e-04-2.32404665e-20j]]
[[ 1.72286133e-04+4.60574165e-20j -1.25004513e-02+3.99939602e-03j]
 [-1.25004513e-02-3.99939602e-03j  9.99827714e-01+4.01623319e-18j]]


In this case, the ideal measurement correspond to single qubit projective measurement in computational basis:

In [8]:
ideal_measurement = povmtools.computational_projectors(d=2)
for Pi in ideal_measurement:
    print(Pi)

[[1. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 1.]]


Let us calculate operational distance between estimator and perfect measurement:

In [9]:
Dop=povmtools.operational_distance_POVMs(ml_povm_estimator,ideal_measurement)
print(Dop)

0.013125781238545442


We see that it is quite small number, but not exactly 0. The reason is that in the execution of circuits we **sample** from ideal distributions, hence our results are affected by statistical fluctuations.

To see that, let us increase number of shots by factor ~16, which should decrease statistical errors roughly by factor ~4.

In [10]:
backend = Aer.get_backend('qasm_simulator')

#define number of measurement repetitions
shots_number = 2000*16

QDT_job_big_statistics = execute(test_circuits, backend=backend, shots=shots_number)

results_big_statistics = QDT_job_big_statistics.result()

#get estimator
DTF_big_statistics = DetectorTomographyFitter()
ml_povm_estimator_big_statistics = DTF.get_maximum_likelihood_povm_estimator([results_big_statistics], test_probe_kets)

In [11]:
#calculate distance
Dop_big_statistics=povmtools.operational_distance_POVMs(ml_povm_estimator_big_statistics,ideal_measurement)
print(Dop_big_statistics)

0.0026344634483292766


## Multiple qubit case

In the mutli-qubit QDT scenario, the only difference is in defining set of indices.

In [12]:
# choose qubit indices
test_qubit_indices = [3, 1]

In all methods, we use convention that **we number qubits in the Hilbert space in ascending order**. Hence in this example the qubits will be sorted such so qubit no. 1 is first and qubit no. 3 is second in the two-qubit Hilbert space.

In [13]:
#Define probe kets. Note that we define them only for single qubit, in the same way as before. 
test_probe_kets = povmtools.pauli_probe_eigenkets

#get circuits
test_circuits_multiple_qubits = detector_tomography_circuits(test_qubit_indices, test_probe_kets)

#define number of repetitions
shots_number=8192

#run job on simulator
backend = Aer.get_backend('qasm_simulator')
QDT_job_multiple_qubits = execute(test_circuits_multiple_qubits, backend=backend, shots=shots_number)
results_multiple_qubits = QDT_job_multiple_qubits.result()

#get estimator
DTF = DetectorTomographyFitter()
ml_povm_estimator_multiple_qubits = DTF.get_maximum_likelihood_povm_estimator([results_multiple_qubits], test_probe_kets)

Again, let's compare it with perfect measurement:

In [14]:
#get ideal measurement in computational basis
ideal_measurement_multiple_qubits = povmtools.computational_projectors(d=2**len(test_qubit_indices))

#calculate distance
Dop_multiple_qubits=povmtools.operational_distance_POVMs(ml_povm_estimator_multiple_qubits,ideal_measurement_multiple_qubits)
print(Dop_multiple_qubits)

0.0034880281697436187


### Joining sets of qubits

In general, performing tomography on the whole device is infeasible. For 15-qubit IBM's device it would require at least 4^15~1 073 741 824 quantum circuits. It is not possible to run so many circuits on any device (and even if it were possible, the sampling complexity would be daunting).

However, under constraints on the locality of the correlations between errors in the device, it is possible to perform QDT. 

To see that, let us consider the layout of the 5q devices with T-shape connectivity (a lot of such devices is currently available in IBM Q Experience). Let us assume that from some other experiments, we have evidence that errors on qubits 0 and 2 do not depend on the measurements of qubits 1,3,4, but they do depend on each other. This is to be understood that doing tomography separately on q0 and q2 and joining the results give different results than doing it simultaneously on q0q2. Similarly, qubits 1,3,4 also depend on each other. 

In that case, we can perform two tomographies:

In [15]:
# choose qubit indices
qubit_indices0, qubit_indices1 = [0,2] , [1,3,4]


#Define probe kets. Note that we define them only for single qubit, in the same way as before. 
probe_kets = povmtools.pauli_probe_eigenkets

#get circuits
circuits0, circuits1= detector_tomography_circuits(qubit_indices0, test_probe_kets), detector_tomography_circuits(qubit_indices1, test_probe_kets)

#define number of repetitions
shots_number=8192

#run first job on simulator and get results
backend = Aer.get_backend('qasm_simulator')
QDT_job0 = execute(circuits0, backend=backend, shots=shots_number)
results0 = QDT_job0.result()

DTF = DetectorTomographyFitter()
POVM0 = DTF.get_maximum_likelihood_povm_estimator([results0], test_probe_kets)

#run second job on simulator and get results
backend = Aer.get_backend('qasm_simulator')
QDT_job1 = execute(circuits1, backend=backend, shots=shots_number)
results1 = QDT_job1.result()

DTF = DetectorTomographyFitter()
POVM1 = DTF.get_maximum_likelihood_povm_estimator([results1], test_probe_kets)

Now we have two tomographies of 4- and 8-dimensional POVMs. We would like to join them into big POVM on 32-dimensional Hilbert space:

In [16]:
#define list of POVMs:
list_POVMs = [POVM0, POVM1]

#define list of associated qubit indices
list_indices = [qubit_indices0, qubit_indices1]

DTF = DetectorTomographyFitter()


big_POVM=DTF.join_povms(list_POVMs,list_indices);

This POVM is quite big, so it is impractical to print whole matrices. However, to check if the order of effects is correct, we may print only *i*th diagonal element of *i*th effect. Each such element should be ~1. 

If order would have been incorrect, we would sometimes obtain values close to 0:

In [17]:
for i in range(2**(sum([len(x) for x in list_indices]))):
    m_i = big_POVM[i]
    
    #take real part to not print 0j
    print(np.real(np.round(m_i[i,i],5)))

0.99998
0.99998
0.99998
0.99997
0.99998
0.99998
0.99998
0.99998
0.99997
0.99997
0.99998
0.99997
0.99997
0.99997
0.99998
0.99997
0.99998
0.99998
0.99998
0.99998
0.99998
0.99998
0.99998
0.99997
0.99997
0.99997
0.99998
0.99997
0.99997
0.99997
0.99998
0.99997
